<a href="https://colab.research.google.com/github/IGVP/Python_IGVP/blob/main/Lesson_3_Potekhin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
# Импортируем нужные бибилиотеки
import pandas as pd

In [91]:
# Загружаем в Python с помощью Pandas все датасеты
user_table = pd.read_csv('user_table.csv')
# Просматриваем кол-во строк и столбцов каждого датасета
print(user_table.shape)
# Просматриваем первые 3 строкb датасета
user_table.head(3)

(90400, 4)


,user_id,date,device,sex
0,450007,2015-02-28,Desktop,Female
1,756838,2015-01-13,Desktop,Male
2,568983,2015-04-09,Desktop,Male


In [92]:
home_page = pd.read_csv('/content/home_page_table.csv')
print(home_page.shape)
home_page.head(3)

(90400, 2)


,user_id,page
0,313593,home_page
1,468315,home_page
2,264005,home_page


In [93]:
search_page = pd.read_csv('/content/search_page_table.csv')
print(search_page.shape)
search_page.head(3)

(45200, 2)


,user_id,page
0,15866,search_page
1,347058,search_page
2,577020,search_page


In [94]:
payment_page = pd.read_csv('/content/payment_page_table.csv')
print(payment_page.shape)
payment_page.head(3)

(6030, 2)


,user_id,page
0,253019,payment_page
1,310478,payment_page
2,304081,payment_page


In [95]:
payment_confirmation = pd.read_csv('/content/payment_confirmation_table.csv')
print(payment_confirmation.shape)
payment_confirmation.head(3)

(452, 2)


,user_id,page
0,123100,payment_confirmation_page
1,704999,payment_confirmation_page
2,407188,payment_confirmation_page


In [96]:
# Меняем название одинаковой колонки в каждом датасете
home_page = home_page.rename(columns = {'page':'home_page'})
search_page = search_page.rename(columns = {'page':'search_page'})
payment_page = payment_page.rename(columns = {'page':'payment_page'})
payment_confirmation = payment_confirmation.rename(columns = {'page':'payment_confirmation'})

In [97]:
# Склеиваем датасет по user_id
df = user_table.merge(home_page, on = 'user_id', how = 'left').merge(search_page, on = 'user_id', how = 'left').\
merge(payment_page, on = 'user_id', how = 'left').merge(payment_confirmation, on = 'user_id', how = 'left')
print(df.shape)
df.head(3)

(90400, 8)


,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,home_page,NaN,NaN,NaN
1,756838,2015-01-13,Desktop,Male,home_page,NaN,NaN,NaN
2,568983,2015-04-09,Desktop,Male,home_page,search_page,NaN,NaN


In [98]:
# Перекодируем названия событий в 0/1 с помощью лямбда-функции
df['home_page'] = df['home_page'].apply(lambda x: 1 if x == 'home_page' else 0)
df['search_page'] = df['search_page'].apply(lambda x: 1 if x == 'search_page' else 0)
df['payment_page'] = df['payment_page'].apply(lambda x: 1 if x == 'payment_page' else 0)
df['payment_confirmation'] = df['payment_confirmation'].apply(lambda x: 1 if x == 'payment_confirmation_page' else 0)
df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,1,0,0,0
1,756838,2015-01-13,Desktop,Male,1,0,0,0
2,568983,2015-04-09,Desktop,Male,1,1,0,0
3,190794,2015-02-18,Desktop,Female,1,1,0,0
4,537909,2015-01-15,Desktop,Male,1,0,0,0


In [108]:
# Проверяем вхождение мужчин и женщин
df['sex'].value_counts()

Male      45325
Female    45075
Name: sex, dtype: int64

In [115]:
# Агрегируем данные: подсчитываем общее количество пользователей и сколько из них дошли до  страницы покупки
df['payment_page'].agg({'users':'count', 'clients':'sum'}).reset_index()

,index,payment_page
0,users,90400
1,clients,6030


1. Построить сегментированную воронку конверсии пользователей интернет-магазина по полу

In [101]:
from pandas.core.groupby import groupby
# Подготовим группировку данных по совершению событий на сайте в разрезе пола пользователя
dfg = df.groupby('sex')[['home_page','search_page','payment_page','payment_confirmation']].\
    sum().unstack('sex').unstack('sex').reset_index()
# Уберем ненужный заголовок
dfg.columns.name = None
# Переименуем index в page
dfg = dfg.rename(columns = {'index':'Page'})
dfg.head()

,Page,Female,Male
0,home_page,45075,45325
1,search_page,22676,22524
2,payment_page,3100,2930
3,payment_confirmation,241,211


In [116]:
# Импортируем графическую библиотеку Plotly 
from plotly import graph_objects as go

# Настраиваем скрипт для построения воронки конверсий нашими параметрами
fig = go.Figure(go.Funnel(
    y = dfg['Page'],
    x = dfg['Male'],
    textinfo = "value+percent previous",
    name = 'Мужчины'
))

fig.add_trace(go.Funnel(
    y = dfg['Page'],
    x = dfg['Female'],
    textinfo = "value+percent previous",
    name = 'Женищины'
))

fig.update_layout(
    title = 'Воронка конверсий пользователей интернет-магазина по полу.',
    title_x=0.5,
    autosize=False,
    width=1000,
    height=500)

fig.show()

Создать 2 новые фичи на основе колонки “date”: месяц и день недели.

In [103]:
# Приводим поле 'date' к занчению даты и вытягиваем название месяца и дня недели
# Добавляем поля в исходный датасет df
df['month'] = pd.to_datetime(df['date']).dt.month_name()
df['dayname'] = pd.to_datetime(df['date']).dt.day_name()
# Смотрим результат
df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation,month,dayname
0,450007,2015-02-28,Desktop,Female,1,0,0,0,February,Saturday
1,756838,2015-01-13,Desktop,Male,1,0,0,0,January,Tuesday
2,568983,2015-04-09,Desktop,Male,1,1,0,0,April,Thursday
3,190794,2015-02-18,Desktop,Female,1,1,0,0,February,Wednesday
4,537909,2015-01-15,Desktop,Male,1,0,0,0,January,Thursday


Определить самый топовый по продажам месяц и день недели с помощью базовых методов агрегации (sum или count).


Опеределим топовый месяц, используем crosstab и pivot_table

In [104]:
# Используем функцию crosstab
pd.crosstab(index = df['month'], columns = df['payment_confirmation']).sort_values(by = 1, ascending = False)

payment_confirmation,0,1
month,,
January,22411,189
February,22427,173
April,22554,46
March,22556,44


In [105]:
# Используем pivot_table
pd.pivot_table(df, index = 'month', columns = 'payment_confirmation',\
               values = 'user_id', aggfunc = 'count').sort_values(by = 1, ascending = False)

payment_confirmation,0,1
month,,
January,22411,189
February,22427,173
April,22554,46
March,22556,44


In [106]:
# Вариант с groupby
df.groupby('month')['payment_confirmation'].sum().reset_index().sort_values(by = 'payment_confirmation', ascending = False)

,month,payment_confirmation
2,January,189
1,February,173
0,April,46
3,March,44


Опеределим топовый день недели, используем groupby, pivot_table и crosstab

In [107]:
# используем groupby
df.groupby(['dayname', 'payment_confirmation'])['user_id'].count().unstack('payment_confirmation')\
            .sort_values(by = 1, ascending = False)

payment_confirmation,0,1
dayname,,
Monday,12878,77
Saturday,12731,70
Tuesday,12628,69
Thursday,13378,66
Sunday,12860,62
Friday,12696,54
Wednesday,12777,54


In [ ]:
# Используем pivot_table
pd.pivot_table(df, index = 'dayname', columns = 'payment_confirmation', \
               values = 'user_id', aggfunc = 'count').sort_values(by = 1, ascending = False)

payment_confirmation,0,1
dayname,,
Monday,12878,77
Saturday,12731,70
Tuesday,12628,69
Thursday,13378,66
Sunday,12860,62
Friday,12696,54
Wednesday,12777,54


Топовый месяц - январь.

Топовый день недели - понедельник.

4. Выяснить, в какой день недели лучше покупают женщины, а в какой мужчины с помощью одного из методов построения сводных таблиц (groupby, pivot_table или crosstab).

In [88]:
# Используем метод groupby
df.groupby(['sex', 'dayname'])['payment_confirmation'].sum().reset_index().\
            sort_values(by = 'payment_confirmation', ascending = False)

,sex,dayname,payment_confirmation
1,Female,Monday,43
5,Female,Tuesday,40
2,Female,Saturday,36
3,Female,Sunday,35
11,Male,Thursday,35
8,Male,Monday,34
9,Male,Saturday,34
4,Female,Thursday,31
6,Female,Wednesday,30
12,Male,Tuesday,29


In [78]:
# Используем метод pivot_table
df.pivot_table(index = ['sex','dayname'], values = 'payment_confirmation', aggfunc = 'sum').unstack('sex')

payment_confirmation     
sex                     Female Male
dayname                            
Friday                      26   28
Monday                      43   34
Saturday                    36   34
Sunday                      35   27
Thursday                    31   35
Tuesday                     40   29
Wednesday                   30   24

In [89]:
# Используем функцию crosstab
pd.crosstab(index = [df['sex'], df['dayname']], columns = df['payment_confirmation'])

payment_confirmation     0   1
sex    dayname                
Female Friday         6302  26
       Monday         6357  43
       Saturday       6308  36
       Sunday         6471  35
       Thursday       6697  31
       Tuesday        6370  40
       Wednesday      6329  30
Male   Friday         6394  28
       Monday         6521  34
       Saturday       6423  34
       Sunday         6389  27
       Thursday       6681  35
       Tuesday        6258  29
       Wednesday      6448  24

Женищины лучше покупают в понедельник.

Мужчины больше покупают в четверг.



